In [1]:
import sys
import pandas as pd
import numpy as np
print(sys.version)
print("imported pandas and numpy")

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 08:22:19) [Clang 14.0.6 ]
imported pandas and numpy


# File contents
The raw data were from ../data/*
Files include:
1. UNAZ-03-24VWCLP+ BRAIN DATA TABLES (GLOBAL).XLSX: Global metabolite
2. UNAZ-03-24VWCLP+ BRAIN HEATMAP (CLP).XLSX
3. UNAZ-03-24VWCLP+ BRAIN HEATMAP (GLOBAL).XLSX
4. UNAZ-03-24VWCLP+ BRAIN SIX TAB FILE (CLP DATA).XLSX
5. UNAZ-03-24VWCLP+ PLASMA DATA TABLES (GLOBAL).XLSX
6. UNAZ-03-24VWCLP+ PLASMA HEATMAP (CLP).XLSX
7. UNAZ-03-24VWCLP+ PLASMA HEATMAP (GLOBAL).XLSX
8. UNAZ-03-24VWCLP+ PLASMA SIX TAB FILE (CLP DATA).XLSX
9. UNAZ-03-24VWCLP+ REPORT_16DEC24.DOCX

From the report, "An equal volume of all plasma samples, and an equal mass of all cortex samples were prepared for analysis on each platform, therefore no additional normalization of the data was performed prior to statistical analysis". 
So this processing file only deal with the meta data, extract the normalized expressions of each metabolites

## Data processing from the report:
*Batch Normalization*: To remove batch variability, for each metabolite, the values in the experimental samples are divided by the median of those samples in each instrument batch, giving each batch and thus the metabolite a median of one.  

*Volumn Normalization*: For each sample, the Batch-normalized data is divided by the value of the normalizer. Since "An equal volume of all plasma samples, and an equal mass of all cortex samples were prepared for analysis on each platform", the value of the normalizer is 1. Then each metabolite is re-scaled to have median = 1 (divide the new values by the overall median for each metabolite). 

*Missing Value imputation*:For each metabolite, the minimum value across all batches in the median scaled data is imputed for the missing values

*log transformation*:The Batch-norm-Imputed Data (or protein, volume, etc. normalized data if applicable) is transformed using the natural log. Metabolomic data typically displays a log-normal distribution, therefore, the log-transformed data is used for statistical analyses.


# Quant 500 Materials and Methods
Kit Description:
The MxP® Quant 500 assay quantifies over 500 metabolites from twenty-six analyte groups: 

|UPLC Analytes|FIA Analytes|
|---|---|
|alkaloids|acylcarnitines
|amine oxides|lysophosphatidylcholines
|amino acids|phosphatidylcholines
|amino acids related		|sphingomyelins
|bile acids|ceramides
|biogenic amines|dihydroceramides
|carbohydrates and related	|hexosylceramides
|carboxylic acids|dihexosylceramides
|cresols|trihexosylceramides
|fatty acids|cholesteryl esters
|hormones and related|diglycerides
|indoles and derivatives|triglycerides
|nucleobases and related|	
|vitamins and cofactors|

The Q500 kit includes all requisite calibration standards, internal standards, and QC samples.  The use of these standards according to the detailed analysis protocol which was validated in Biocrates’ lab in Austria assures assay harmonization and standardization within a project, across projects, and across laboratories.  Selective analyte detection is accomplished by use of a triple quadrupole tandem mass spectrometer operated in Multiple Reaction Monitoring (MRM) mode in which specific precursor to product ion transitions are measured for every analyte and stable isotope labeled internal standard.


## Data file description

### Table 2. Key to the Analyte Status Columns Used in the 6030 Data Tables Valid
|Status Value|Description|
|---|---|
|Valid|Calculated concentrations are based on a standard curve of the analyte listed.  (Not all analytes are contained in the calibration standards provided by Biocrates.  Therefore, results for some analytes will be coded as Semi-Quantitative.)|
|LowestCS|The value is greater than the LOD but less than the Biocrates-defined lowest calibration standard.  These values should be considered reliable except for analytes for which the %CV observed for the pooled sample was greater than 25%.|
|HighestCS|Greater than the highest calibration standard|
|`<`LOD | Less than the Lower Limit of Detection. The LODs were given by Biocrates for the UPLC analytes not determined using FDA guidelines for a Waters TQ-XS or Sciex 6500+ mass spectrometer.  FIA LODs are calculated for each plate using the Blank samples with Internal Standard.|
|Internal Standard out of range|The internal standard peak area for this analyte was outside of the normal range. (Not all analytes have internal standards.)|
|No Interception or NA|No peak was detected in the chromatogram at the appropriate retention time for this analyte.  For statistical analyses apply the same value used for `<`LOD|
|NA (Metabolism Indicator Status)|Incomplete Metabolism Indicator|

### Table 3. Key to the Analyte Classifications Used in the 6030 Data Tables
|Value|Description|
|---|---|
|Quantitative|Validation criteria fulfilled.  Biocrates have verified the precision (CV) and accuracy in their lab.  For UPLC analytes calculated concentrations are based on a seven-point standard curve of the analyte listed.  (Not all analytes are contained in the calibration standards provided by Biocrates.  Therefore, results for some analytes will not be coded as Quantitative.)  For FIA analytes concentrations are based on a single-point calibration.|
|Quantitative with Restrictions|Results are “Quantitative” but have poorer precision (CV=15-30%) and/or poorer accuracy (accuracy=15-30%)|
|Relative Quantitative|FIA analytes only:  precise (CV<15%) but accuracy not verified.  Internal one-point calibration.|
|Not Validated|Analyte not tested during validation|

### Table 4. Identification of Sample Types contained within the data tables
|Sample Identification|Customer Sample Identification|Number Extracted on Each Plate|Number of Injections/Extraction|Purpose|
|---|---|---|---|---|
|101|6030 Plasma SPQC|1|3|Pool of 76 plasma study samples (from Plate 1). QC representative of the study samples used to measure sample preparation and UPLC/MS/MS variability.  Can also be used for batch correction or normalization.|
|103|6030 Brain SPQC|1|3|Pool of 58 brain study samples. QC representative of the study samples used to measure sample preparation and UPLC/MS/MS variability.  Can also be used for batch correction or normalization.|
|104|6030 Liver SPQC|1|3|Pool 57 liver study samples. QC representative of the study samples used to measure sample preparation and UPLC/MS/MS variability.  Can also be used for batch correction or normalization.|
|102|NIST SRM 1950|2|1|Human plasma pool supplied by National Institutes of Science and Technology as a <b>Standard References Material</b>. QC to measure sample preparation and UPLC/MS/MS variability.|
|Numbers in Column D of data table|Identifiers on sample tubes|1|1|Customer Samples|

# Data Processing

In [2]:
df_18m = pd.read_excel("../data/APOE tissues for metabolomics.xlsx",sheet_name=0,header=0,dtype={'Animal_NAME':str,'Necropsy_Age_months':float})
df_18m.rename(columns={'Animal_NAME':'SampleID','Genotype':'Apoe','Necropsy_Age_months':'Age'},inplace=True)
df_18m=df_18m[['SampleID','Sex','Apoe','Age','Fur_Condition_Notes','Date_of_Birth','Necropsy_Date']]
df_18m[['SampleID']]='S'+df_18m[['SampleID']]
print(df_18m.iloc[1:5,:])

  SampleID     Sex   Apoe        Age             Fur_Condition_Notes  \
1     S864    Male  APOE4  18.779754      ok, hair loss on underside   
2     S870  Female  APOE4  18.779754  bad, lots of fat around organs   
3     S947  Female  APOE3  18.615595                            good   
4     S948  Female  APOE3  18.615595                            good   

  Date_of_Birth  Necropsy_Date  
1    2019-02-09          44077  
2    2019-02-09          44077  
3    2019-02-19          44082  
4    2019-02-19          44082  


In [3]:
print(df_18m.columns)

Index(['SampleID', 'Sex', 'Apoe', 'Age', 'Fur_Condition_Notes',
       'Date_of_Birth', 'Necropsy_Date'],
      dtype='object')


In [4]:
dataFileName="../data/6030 Q500 Brain Data.xlsx"

# Data Processing
We recommend data preprocessing which includes the following steps. (This preprocessing has been performed in the Preprocessed Log2 tab.)  First, analytes with a high percentage of missing values were removed from the dataset.  A standard cutoff for a small sample set was set to >50% of the data is missing, the analyte was removed.  It should be noted that in a case-control study with severe metabolite dysregulation or samples from different species, a high level of missing values may be observed in some samples.  Second, the remaining missing values were filled in with half the limit of detection (LOD/2). This is appropriate because the main cause of ‘missing data’ in the targeted metabolomics workflows is very low concentrations of the analyte. Next, analytes in which the calculated %CV for the SPQC was >30% were removed. The final preprocessing step recommended was to log2 transform the data. This is recommended prior to hypothesis testing because most metabolomics data is log-normal.

Rows 4 through 9 list the Lower Limit of Detection (LOD) values calculated by MetIDQ for FIA per plate, Lowest Calibration Standard, and Highest Calibration Standard for each FIA analyte, respectively.  Rows 10 through 12 list the Lower Limit of Detection values calculated by METIDQ, Lowest Calibration Standard, and Highest Calibration Standard for each UPLC analyte, respectively.  Analyte Classifications (see Table 3 in the Appendix) list the validation status of each analyte in row 14. 

In [5]:
df1=pd.read_excel(dataFileName,sheet_name=0,header=0,skiprows=1,usecols="K:OC")
#print(analytes_meta)
col_names=df1.columns  
analytes_meta=df1.iloc[0:12,col_names.str.contains("Status")==False]

In [6]:
analytes_meta

,Injection Number,C0,C2,C3,C3-DC (C4-OH),C3-OH,C3:1,C4,C4:1,C5,...,DG(16:0_18:1),DG(16:0_18:2),DG(16:0_20:0),DG(16:0_20:3),DG(16:0_20:4),DG(16:1_18:0),DG(16:1_18:1),DG(16:1_18:2),DG(16:1_20:0),DG(17:0_17:1)
0,Class,Acylcarnitines,Acylcarnitines,Acylcarnitines,Acylcarnitines,Acylcarnitines,Acylcarnitines,Acylcarnitines,Acylcarnitines,Acylcarnitines,...,Diacylglycerols,Diacylglycerols,Diacylglycerols,Diacylglycerols,Diacylglycerols,Diacylglycerols,Diacylglycerols,Diacylglycerols,Diacylglycerols,Diacylglycerols
1,FIA LOD Plate 1 (1035442711-1) [µM],1.52,0.117,0.0018,0.0591,0.171,0.014,0.0621,0.01,0.0057,...,0.142,0.211,0.501,0.212,0.265,0.471,0.273,0.265,0.25,0.148
2,FIA LOD Plate 2 (1035442778-1) [µM],1.91,0.0844,0.01,0.0236,0.0718,0.01,0.0333,0.01,0.01,...,0.0982,0.0067,0.12,0.106,0.123,0.01,0.857,0.0854,0.0174,0.009
3,FIA LOD Plate 3 (1035442759-1) [µM],1.65,0.139,0.0113,0.0155,0.105,0.127,0.0384,0.01,0.0028,...,0.0197,0.0814,0.0894,0.157,0.188,0.01,0.119,0.0726,0.01,0.0623
4,FIA LOD Plate 4 (1035442730-1) [µM],1.85,0.124,0.0019,0.0765,0.221,0.0249,0.0649,0.01,0.0103,...,0.308,0.283,0.681,0.399,0.443,0.35,1.09,0.221,0.402,0.197
5,FIA Lowest CS [µM],5,0.9,0.9,0,0.1,0,0.03,0,0.03,...,0,0,0,0,0,0.2,0,0,0,0
6,FIA Highest CS [µM],120,20,4,0,1,0,6,0,6,...,0,0,0,0,0,1,0,0,0,0
7,UPLC LOD [µM],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,UPLC Lowest CS [µM],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,UPLC Highest CS [µM],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import re
df2=pd.read_excel(dataFileName,sheet_name=0,header=0,skiprows=1,usecols="A:J",
                            dtype = str)
sample_start,sample_end=0,0#Start from row 12, end with 
spqc_start,spqc_end=0,0
srm_start,srm_end=0,0
tip=0
for i,sampletype in enumerate(df2["Sample Type"].isnull()):
    if tip==0 and sampletype==False:
        sample_start=i
        tip=1
    elif tip==1 and sampletype==True:
        sample_end=i
        tip=2
    elif tip==2 and sampletype==False:
        srm_start,tip=i,3
    elif tip==3 and sampletype==True:
        srm_end,tip=i,4
    elif tip==4 and sampletype==False:
        spqc_start,tip=i,5
    elif tip==5 and sampletype==True:
        spqc_end,tip=i,6
if tip!=6:
    spqc_end=i
#Sample meta
sample_meta=df2.iloc[sample_start:sample_end,]
sample_meta=sample_meta.astype({'Well Position':'int',
                                'Run Number':'int'})
sample_meta=sample_meta.rename(columns={"Customer Sample Identification":"SampleID"})
sample_meta["SampleID"]=["S"+re.sub('\s.*$','',x) for x in sample_meta["SampleID"].values]
#Data df
data_df=df1.iloc[sample_start:sample_end,col_names.str.contains("Status")==False]
data_df = data_df.iloc[:,1:]
data_status=df1.iloc[sample_start:sample_end,col_names.str.contains("Status")==True]
#srm df
srm_df=df1.iloc[srm_start:srm_end,col_names.str.contains("Status")==False]
srm_df = srm_df.iloc[:,1:]
srm_status=df1.iloc[srm_start:srm_end,col_names.str.contains("Status")==True]
#spqc df
spqc_df=df1.iloc[spqc_start:spqc_end,col_names.str.contains("Status")==False]
spqc_df = spqc_df.iloc[:,1:]
spqc_status=df1.iloc[spqc_start:spqc_end,col_names.str.contains("Status")==True]

In [8]:
sample_start,sample_end,srm_start,srm_end,spqc_start,spqc_end

(12, 70, 71, 79, 82, 94)

In [9]:
#This part should deal with processing xlsx, extract the following
#1.Data
#2.Sample information (sample metadata)
#3.Metabolyte metadata
#4.The internal standard curves
from typing import List
import re
def process_xlsx(excelF:str,sheetnumber:int,statustag=False)-> List[pd.DataFrame]:
    #Load data frames
    df=pd.read_excel(excelF,sheet_name=sheetnumber,header=0,skiprows=1,nrows=1)
    df1=pd.read_excel(excelF,sheet_name=sheetnumber,header=0,skiprows=1,usecols=[i for i in range(10,df.shape[1])])
    df2=pd.read_excel(excelF,sheet_name=sheetnumber,header=0,skiprows=1,usecols=[i for i in range(0,10)],dtype = str)
    #Get analyte meta first
    col_names=df1.columns  
    analytes_meta=df1.iloc[0:12,col_names.str.contains("Status")==False]
    #Get Sample information
    sample_start,sample_end=0,0#Start from row 12, end with 
    spqc_start,spqc_end=0,0
    srm_start,srm_end=0,0
    tip=0
    for i,sampletype in enumerate(df2["Sample Type"].isnull()):
        if tip==0 and sampletype==False:
            sample_start=i
            tip=1
        elif tip==1 and sampletype==True:
            sample_end=i
            tip=2
        elif tip==2 and sampletype==False:
            srm_start,tip=i,3
        elif tip==3 and sampletype==True:
            srm_end,tip=i,4
        elif tip==4 and sampletype==False:
            spqc_start,tip=i,5
        elif tip==5 and sampletype==True:
            spqc_end,tip=i,6

    if tip==1:
        sample_end=i
    elif tip ==3:
        srm_end=i
    elif tip ==5:
        spqc_end=i
    #Sample meta
    sample_meta=df2.iloc[sample_start:sample_end,]
    sample_meta=sample_meta.astype({'Well Position':'int',
                                    'Run Number':'int'})
    sample_meta=sample_meta.rename(columns={"Customer Sample Identification":"SampleID"})
    sample_meta["SampleID"]=["S"+re.sub('\s.*$','',x) for x in sample_meta["SampleID"].values]
    #Data df
    data_df=df1.iloc[sample_start:sample_end,col_names.str.contains("Status")==False]
    data_df = data_df.iloc[:,1:]
    data_status=df1.iloc[sample_start:sample_end,col_names.str.contains("Status")==True]
    #srm df
    srm_df=df1.iloc[srm_start:srm_end,col_names.str.contains("Status")==False]
    srm_df = srm_df.iloc[:,1:]
    srm_status=df1.iloc[srm_start:srm_end,col_names.str.contains("Status")==True]
    #spqc df
    spqc_df=df1.iloc[spqc_start:spqc_end,col_names.str.contains("Status")==False]
    spqc_df = spqc_df.iloc[:,1:]
    spqc_status=df1.iloc[spqc_start:spqc_end,col_names.str.contains("Status")==True]
    if statustag:
        return analytes_meta,sample_meta,data_df,data_status,srm_df,srm_status,spqc_df,spqc_status
    else:
        return analytes_meta,sample_meta,data_df,srm_df,spqc_df



## Process the raw data

In [10]:
analytes1_meta,sample1_meta,data1_df,data1_status,srm1_df,srm1_status,spqc1_df,spqc1_status=process_xlsx(dataFileName,0,statustag=True)


## Dilution Factor Applied

For analysis of the brain and liver samples, worksheet “Dilution Factor Applied” is the worksheet to use if you plan on performing your own preprocessing of the data to remove analytes with high levels of missing values, imputing missing values, and filtering based on %CV. The data on this worksheet has been corrected for the sample volume that was used. (The μM concentration and LOD data corrected with a dilution factor of 0.333 to reflect the extraction of 30 μL of each brain homogenate instead of 10 μL.)

In [11]:
analytes2_meta,sample2_meta,data2_df,srm2_df,spqc2_df=process_xlsx(dataFileName,2)

## Remove Greater than 50 LOD

Compounds with >50% missing values or not detectable were removed.  


In [12]:
analytes3_meta,sample3_meta,data3_df,srm3_df,spqc3_df=process_xlsx(dataFileName,3)

In [13]:
#analytes1_meta
#sample1_meta
#data1_df
#data1_status
#srm1_df
#srm1_status
#spqc2_df.dtypes

#tmp=data3_df.columns[data3_df.columns.isin(data2_df.columns)==False]
tmp=data2_df.columns[data2_df.columns.isin(data3_df.columns)==False]
data2_rm=data2_df[tmp]
tmp2=data3_df.columns[data3_df.columns.isin(data2_df.columns)==True]
data2_keep=data2_df[tmp2]

In [14]:
tmp
#347 were missing in data3
import numpy as np
testrmLOD=[np.count_nonzero(np.logical_or(data2_rm.iloc[:,i].values=='<LOD', data2_rm.iloc[:,i].isnull().values))/data2_rm.shape[0] for i in range(data2_rm.shape[1])]

In [15]:
testrmLOD=np.array(testrmLOD)

In [16]:
np.count_nonzero(testrmLOD>0.5)/len(testrmLOD)

1.0

In [17]:
testkeepLOD=np.array(
    [np.count_nonzero(np.logical_or(data2_keep.iloc[:,i].values=='<LOD', data2_keep.iloc[:,i].isnull().values))/data2_keep.shape[0] 
    for i in range(data2_keep.shape[1])])

In [18]:
np.count_nonzero(testkeepLOD>0.5)/len(testkeepLOD)

0.0

In [19]:
data2_keep.columns[testkeepLOD>0]

Index(['Trigonelline', 'Orn', 'PAG', 'GLCAS', 'TCA', 'Dopamine', 'AconAcid',
       'DiCA(14:0)', 'Suc', 'Cer(d16:1/20:0)',
       ...
       'Cit Synthesis', 'OTC Deficiency (NBS)', 'Polyamine Synthesis',
       'Putrescine Synthesis', 'GABR', 'Orn Synthesis', 'Glutaminolysis Rate',
       'Sum of DH-Cer', 'Sum of LCFA-DH-Cer', 'Sum of VLCFA-DH-Cer'],
      dtype='object', length=125)

## Remove Greater than 30 SPQC
analytes/compounds with calculated %CVs for the SPQC (sample 101) >30% were removed. Values `<`LOD were replaced with LOD/2. 
From data3:

1. Calculate %CV for each compound only based on spqc data.
2. Remove compounds with %CV > 30 or no CV available

In [20]:
filter1_spqc = [np.count_nonzero(np.logical_or(spqc3_df.iloc[:,i].isnull(),spqc3_df.iloc[:,i].values=='<LOD'))>0 for i in range(spqc3_df.shape[1])]

In [21]:
spqc3_df.columns[filter1_spqc]

Index(['5-AVA', 'Orn', 'DiCA(14:0)', 'Cer(d16:1/20:0)', 'Cer(d18:1/14:0)',
       'CE(15:1)', 'CE(17:0)', 'CE(18:0)', 'DG(14:1_18:1)', 'DG(16:0_18:2)',
       ...
       'Ratio of SGA to Hexose', 'LDH Activity', 'Sum of DH-Cer',
       'Sum of LCFA-DH-Cer', 'Sum of VLCFA-DH-Cer', 'Sum of SFAs',
       'Ratio of MUFA-PCs to SFA-PCs', 'Sum of PCs (aa)', 'Sum of SFA-PCs',
       'Ratio of PCs (aa) to Choline'],
      dtype='object', length=115)

In [22]:
#CV% > 30
CV_threshold=30
filter2_spqc=[filter1_spqc[i] or np.std(spqc3_df.iloc[:,i],ddof=1)/np.mean(spqc3_df.iloc[:,i])*100 >CV_threshold  for i in range(spqc3_df.shape[1])]

In [23]:
spqc3_df.columns[filter2_spqc==False]

<ipython-input-23-c7115f4fe825>:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  spqc3_df.columns[filter2_spqc==False]


array([], shape=(0, 484), dtype=object)

In [24]:
spqc3_df.columns[~np.array(filter2_spqc)]

Index(['C0', 'C2', 'C3', 'C3-DC (C4-OH)', 'C3-OH', 'C4', 'C4:1', 'C5',
       'C5-DC (C6-OH)', 'C5-M-DC',
       ...
       'Sum of MUFA-PCs', 'Sum of VLCFA-LysoPCs', 'Sum of Purines',
       'Xanthine Synthesis', 'Ratio of SM-OHs to SM-Non OHs',
       'Sum of LCFA-SMs', 'Sum of SM-Non OHs', 'Sum of SM-OHs', 'Sum of SMs',
       'Sum of VLCFA-SMs'],
      dtype='object', length=331)

In [25]:
data4_df_recal=data3_df.iloc[:,~np.array(filter2_spqc)]
srm4_df_recal,spqc4_df_recal=srm3_df.iloc[:,~np.array(filter2_spqc)],spqc3_df.iloc[:,~np.array(filter2_spqc)]

## Replace values `<`LOD
Values `<`LOD were replaced with LOD/2. 
To do so:
- 1.Find i,j data_df.iloc[i,j].values=='<LOD'
- 2.For jth compound, get the info weather it is FIA or UPLC based on their category row
- 3.For i's row, find the plate number 'Plate Bar Code'
- 4.Get LOD values of either FIA_Plate or UPLC from step2 and step3.

FIA Analytes:
- acylcarnitines
- lysophosphatidylcholines
- phosphatidylcholines
- sphingomyelins
- ceramides
- dihydroceramides
- hexosylceramides
- dihexosylceramides
- trihexosylceramides
- cholesteryl esters
- diglycerides
- triglycerides

UPLC Analytes:
- alkaloids				
- amine oxides			
- amino acids			
- amino acids related	
- bile acids				
- biogenic amines		
- carbohydrates and related	
- carboxylic acids			
- cresols					
- fatty acids				
- hormones and related		
- indoles and derivatives	
- nucleobases and related	
- vitamins and cofactors

## Log2 transformation

In [26]:
analytes5_meta,sample5_meta,data5_df,srm5_df,spqc5_df=process_xlsx(dataFileName,5)

In [27]:
analytes6_meta,sample6_meta,data6_df,srm6_df,spqc6_df=process_xlsx(dataFileName,6)

In [28]:
data6_df=data6_df.astype(float)#data6_df.shape

In [29]:
data6_df

,C0,C2,C3,C3-DC (C4-OH),C3-OH,C3:1,C4,C4:1,C5,C5-DC (C6-OH),...,Sum of VLCFA-LysoPCs,Ratio of PCs (aa) to Choline,Sum of Purines,Xanthine Synthesis,Ratio of SM-OHs to SM-Non OHs,Sum of LCFA-SMs,Sum of SM-Non OHs,Sum of SM-OHs,Sum of SMs,Sum of VLCFA-SMs
12,2.647698,0.263034,-3.928695,-3.698998,-1.967664,-2.816037,-4.246966,-5.297136,-4.917025,-4.584963,...,-0.043943,2.104337,6.297681,-4.256126,-7.333516,5.745954,5.946419,0.193246,5.969626,3.142958
13,3.387730,1.694509,-3.298081,-3.023270,-2.031111,-2.839940,-3.283960,-5.297136,-3.822826,-4.237864,...,0.106572,2.341037,6.502500,-4.705257,-7.137965,5.954196,6.149747,0.597730,6.183222,3.419539
14,2.861294,1.150560,-3.602380,-2.954557,-1.965784,-2.799203,-3.822826,-5.216250,-4.782562,-3.906891,...,-0.029146,2.302563,6.080373,-4.431806,-7.349113,5.807355,6.007495,0.242857,6.037089,3.247928
15,2.655352,0.847997,-4.066931,-3.511828,-2.385840,-2.996158,-4.495464,-5.668104,-4.892535,-4.265345,...,0.023847,2.560715,6.321928,-4.573467,-7.287712,5.977280,6.169925,0.464668,6.196397,3.359896
16,2.450661,0.861294,-3.473931,-3.326745,-2.258425,-2.936037,-3.936037,-5.497635,-4.668104,-4.202019,...,0.128733,3.029747,5.403722,-4.312342,-7.438047,6.108524,6.254241,0.393233,6.273018,3.064653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,3.153805,1.635367,-3.210897,-2.857260,-2.009650,-1.009650,-3.479284,-5.308525,-3.857260,-3.871267,...,0.246915,2.180572,6.163230,-4.442222,-7.624747,5.961932,6.108524,0.065802,6.129283,2.963474
77,3.164572,1.611959,-3.197600,-2.843388,-2.027185,-1.029146,-3.473931,-5.265345,-4.050901,-3.885411,...,0.209973,2.212050,6.129283,-4.370838,-7.596550,6.007495,6.156504,0.146221,6.183222,3.047306
78,3.126532,1.616671,-3.233334,-2.772670,-2.085180,-2.792524,-3.452715,-5.323853,-4.302819,-3.878321,...,0.184809,2.201634,6.169925,-4.341293,-7.578054,5.930737,6.087463,0.093109,6.108524,3.005999
79,3.053111,1.454176,-3.317127,-2.928695,-1.910502,-2.753085,-3.479284,-5.239680,-4.421464,-3.996158,...,0.163499,2.035624,6.247928,-4.312342,-7.568894,5.890771,6.044394,0.061200,6.066089,2.963474


In [30]:
data5_df=data5_df.astype(float)

In [31]:
for i in range(data5_df.shape[0]):
    for j in range(data5_df.shape[1]):
        if data5_df.iloc[i,j]<0.0001:
            print(i,j,data5_df.iloc[i,j],data5_df.columns[j])
            #print(data5_df.columns[j])

0 335 9.999999999999999e-05 BABA Synthesis
24 335 9.999999999999999e-05 BABA Synthesis
30 335 6.666666666666667e-05 BABA Synthesis
38 335 9.999999999999999e-05 BABA Synthesis
54 335 6.666666666666667e-05 BABA Synthesis
55 335 6.666666666666667e-05 BABA Synthesis
57 266 0.0 TG(20:3_36:4)


In [32]:
data5_df_log2=np.log2(data5_df)

/Users/yshang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)


In [33]:
data5_df_log2.iloc[57,266]

-inf

# Save Data
get data only for compounds in the list of spqc3_df.columns[~np.array(filter2_spqc)]

In [34]:
#Save Data
data5_df_log2_filter = data5_df_log2[spqc3_df.columns[~np.array(filter2_spqc)]]
data5_df_filter = data5_df[spqc3_df.columns[~np.array(filter2_spqc)]]

In [35]:
data5_df_log2_filter.shape

(58, 331)

In [36]:
data5_df_log2.shape

(58, 382)

In [37]:
sample5_meta["SampleID"].values

array(['S863', 'S864', 'S870', 'S947', 'S948', 'S949', 'S951', 'S956',
       'S957', 'S958', 'S959', 'S960', 'S961', 'S962', 'S963', 'S968',
       'S970', 'S971', 'S974', 'S975', 'S976', 'S977', 'S978', 'S979',
       'S980', 'S981', 'S982', 'S983', 'S984', 'S985', 'S986', 'S987',
       'S988', 'S991', 'S992', 'S993', 'S994', 'S1002', 'S1003', 'S1026',
       'S1027', 'S1028', 'S1029', 'S1033', 'S1034', 'S1035', 'S1036',
       'S1037', 'S1038', 'S1124', 'S1125', 'S1324', 'S1325', 'S1326',
       'S1330', 'S1440', 'S1442', 'S1443'], dtype=object)

In [38]:
data5_df_log2_filter.insert(loc=0, column="SampleID", value=sample5_meta["SampleID"].values)


In [39]:
data5_df_filter.insert(loc=0, column="SampleID", value=sample5_meta["SampleID"].values)


In [40]:
data5_df_log2_filter.to_csv("../data/brain_log2_notscaled.csv")

In [41]:
data5_df_filter.to_csv("../data/brain_notscaled.csv")

In [42]:
data5_df_meta = df_18m.merge(sample5_meta,on="SampleID")

In [43]:
df_18m.iloc[0:5,:]

,SampleID,Sex,Apoe,Age,Fur_Condition_Notes,Date_of_Birth,Necropsy_Date
0,S863,Male,APOE4,18.779754,bad,2019-02-09,44077
1,S864,Male,APOE4,18.779754,"ok, hair loss on underside",2019-02-09,44077
2,S870,Female,APOE4,18.779754,"bad, lots of fat around organs",2019-02-09,44077
3,S947,Female,APOE3,18.615595,good,2019-02-19,44082
4,S948,Female,APOE3,18.615595,good,2019-02-19,44082


In [44]:
data5_df_meta.to_csv("../data/brain_meta.csv")